In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py

import kalepy as kale

import holodeck as holo
from holodeck.constants import MSOL, PC, YR, MPC

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})

In [ ]:
import holodeck.pop_observational
reload(holodeck.pop_observational)

In [ ]:
FNAME = "observational_2mass_galaxy-catalog_extended.npz"
fname = os.path.join(holo._PATH_DATA, FNAME)
# _pop, pop = holodeck.pop_observational.pipeline()
pop = holo.pop_observational.BP_Observational(fname)

In [ ]:
pop.mtot, pop.mrat = holo.utils.mtmr_from_m1m2(pop.mass)
pop.redz = holo.cosmo.a_to_z(pop.scafa)

In [ ]:
data = [np.log10(pop.mtot/MSOL), np.log10(pop.mrat), pop.redz]
weight = pop.weight
idx = (weight > 0.0)
data = [dd[idx] for dd in data]
weight = weight[idx]
reflect = [None, True, [0, None]]
print(weight.size)

# bad = ~np.isfinite(data[-1])
# data[-1][bad] = np.inf
# print(data[-1])
# print(np.percentile(data[-1], [0, 25, 50, 75, 100]))
# breaker()

# for dd in data:
#     print(dd)
#     print(np.percentile(dd, [0, 25, 50, 75, 100]))
    
kde = kale.KDE(data, weights=weight, reflect=reflect)
corner = kale.Corner(kde, rotate=False, labels=['mtot', 'mrat', 'redz'])
corner.plot()
plt.show()

In [ ]:
def create_extended_single_catalog(out_fname):
    fname = "2mass_galaxies.lst"
    fname = os.path.join(holo._PATH_DATA, fname)
    catalog = np.loadtxt(fname, usecols=(1, 2, 3, 4))

    fname = "added_Mks.lst"
    fname = os.path.join(holo._PATH_DATA, fname)
    ext_catalog = np.loadtxt(fname, usecols=(1, 2, 3, 4, 5), skiprows=2)

    k_mag = catalog[:, 3]
    k_mag = np.hstack((k_mag, ext_catalog[:, 4]))

    fname = os.path.join(holo._PATH_DATA, out_fname)
    np.savez(fname, k_mag=k_mag)
    print(f"saved to {fname}")
    return

create_extended_single_catalog()